In [1]:
import tensorflow as tf

### Tensors and Operations


In [2]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
t

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [3]:
t.shape

TensorShape([2, 3])

In [4]:
t.numpy()

array([[1, 2, 3],
       [4, 5, 6]])

In [5]:
print(t[0, 0])
print(t[:, 2])

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor([3 6], shape=(2,), dtype=int32)


In [6]:
t[..., :]

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [7]:
t[..., 1, tf.newaxis]  # keep dimension 1

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [5]])>

In [8]:
t+10

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[11, 12, 13],
       [14, 15, 16]])>

In [9]:
t.__add__(10)  # not in-place

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[11, 12, 13],
       [14, 15, 16]])>

In [10]:
t

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [11]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 1,  4,  9],
       [16, 25, 36]])>

In [12]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

### Tensors and NumPy


In [7]:
import numpy as np

In [14]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1,  4,  9],
       [16, 25, 36]])

### Type Conversions


In [17]:
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: 


In [18]:
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    # defualt values in tensorflow are float32, cannot manipulated with float64
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: 


In [19]:
tf.constant(2.) + tf.constant(50, dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=52.0>

In [20]:
t2 = tf.constant(40, dtype=tf.float64)
tf.constant(2.) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables


In [21]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [22]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [23]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [24]:
v[0, 0].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[42.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [25]:
v.scatter_nd_update(indices=[[0, 0], [1, 0]], updates=[100, 400])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   4.,   6.],
       [400.,  10.,  12.]], dtype=float32)>

In [26]:
try:
    t[1] = [7., 8., 9.]  # cannot update with direct assignment
except TypeError as ex:
    print(ex)

'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment


### Custom Loss Function


In [2]:
def huber_fn(y_true, y_pred):
    """combine MSE smoothing property (at small values) and MAE outliers robustness property (at large values)"""
    error = abs(y_true-y_pred)
    squared = tf.square(error) / 2  # smoothing gradients factor
    # -.5 for connection guaranteed between two losses at the threshold (1 here)
    linear = error - .5
    return tf.where(error < 1, squared, linear)

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

input_shape = X_train.shape[1:]

tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

C:\Users\sayed\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(loss=huber_fn, metrics=['mae'], optimizer='nadam')

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7494 - mae: 1.1371 - val_loss: 0.3474 - val_mae: 0.6522
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: 0.2607 - mae: 0.5681 - val_loss: 0.2553 - val_mae: 0.5383


### Saving/Loading Models with Custom objects


In [10]:
import os
os.makedirs('saved_models', exist_ok=True)

In [11]:
model.save('saved_models/my_model_with_custom_hyper_loss.keras')

Load the model:


In [12]:
model = tf.keras.models.load_model(
    'saved_models/my_model_with_custom_hyper_loss.keras', custom_objects={'huber_fn': huber_fn})

In [13]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2188 - mae: 0.5096 - val_loss: 0.2129 - val_mae: 0.4876
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - loss: 0.1990 - mae: 0.4809 - val_loss: 0.1891 - val_mae: 0.4610


#### Now, We need to control the threshold..


In [14]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [15]:
model.compile(loss=create_huber(2.0), metrics=['mae'], optimizer='nadam')

In [16]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2148 - mae: 0.4689 - val_loss: 0.2084 - val_mae: 0.4504
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 0.2054 - mae: 0.4597 - val_loss: 0.1807 - val_mae: 0.4340


In [17]:
model.save("saved_models/my_model_with_a_custom_loss_threshold_2.keras")

In [18]:
model = tf.keras.models.load_model("saved_models/my_model_with_a_custom_loss_threshold_2.keras",
                                   custom_objects={"huber_fn": create_huber(2.0)})

so the threshold will not be saved with the model, but we must pass it..


In [19]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2000 - mae: 0.4540 - val_loss: 0.1853 - val_mae: 0.4339
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - loss: 0.1959 - mae: 0.4490 - val_loss: 0.1976 - val_mae: 0.4367


now we need to build huber function that saves the threshold with it, we must use sub-classing..


In [20]:
import keras

### Sub-Classing Custom Loss Function with automatically saved hyperparameters


In [21]:
@keras.utils.register_keras_serializable()
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [22]:
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0512 - mae: 1.1438 - val_loss: 0.5086 - val_mae: 0.6718
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - loss: 0.3170 - mae: 0.5816 - val_loss: 0.3527 - val_mae: 0.5571


In [23]:
model.save("saved_models/my_model_with_a_custom_loss_class.keras")

> Note: when using sub-classing, either passing the custom class used as custom objects, or decorate the class with @keras.utils.register_keras_serializable(),


In [24]:
model = tf.keras.models.load_model('saved_models/my_model_with_a_custom_loss_class.keras'
                                   #    ,custom_objects={'HuberLoss': HuberLoss}
                                   )

In [25]:
model.loss

here's, it identify it even without passing it as custom object to load_model function


---


### Custom Activation Functions, Initializers, Regularizers,and Constraints


In [26]:
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))


def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)


def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))


def my_positive_weights(weights):  # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [27]:
layer = tf.keras.layers.Dense(1, activation=my_softplus,
                              kernel_initializer=my_glorot_initializer,
                              kernel_regularizer=my_l1_regularizer,
                              kernel_constraint=my_positive_weights)

Full example using custom functions above


In [28]:
tf.keras.utils.set_random_seed(42)

try:
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                              input_shape=input_shape),
        tf.keras.layers.Dense(1, activation=my_softplus,
                              kernel_initializer=my_glorot_initializer,
                              kernel_regularizer=my_l1_regularizer,
                              kernel_constraint=my_positive_weights)
    ])
    model.compile(loss="mse", optimizer="nadam", metrics=["mae"])
    model.fit(X_train_scaled, y_train, epochs=2,
              validation_data=(X_valid_scaled, y_valid))
    model.save("saved_models/my_model_with_many_custom_parts")
    model = tf.keras.models.load_model(
        "saved_models/my_model_with_many_custom_parts",
        custom_objects={
            "my_l1_regularizer": my_l1_regularizer,
            "my_positive_weights": my_positive_weights,
            "my_glorot_initializer": my_glorot_initializer,
            "my_softplus": my_softplus,
        }
    )
    model.fit(X_train_scaled, y_train, epochs=2,
              validation_data=(X_valid_scaled, y_valid))

except Exception as ex:
    print(ex)

Invalid value for attribute `regularizer`. Expected an instance of `keras.regularizers.Regularizer`, or `None`. Received: regularizer=<function my_l1_regularizer at 0x000001579AE93C70>


here's a problem with directly assignment regular functions, we should do it with sub-classing


In [29]:
# Custom Regularizer
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor=0.01):
        self.factor = factor

    def __call__(self, x):
        return tf.reduce_sum(tf.abs(self.factor * x))

    def get_config(self):
        return {"factor": self.factor}

# Custom Constraint


class MyPositiveWeights(tf.keras.constraints.Constraint):
    def __call__(self, weights):
        return tf.nn.relu(weights)

    def get_config(self):
        return {}

# Custom Initializer


class MyGlorotInitializer(tf.keras.initializers.Initializer):
    def __init__(self):
        super(MyGlorotInitializer, self).__init__()

    def __call__(self, shape, dtype=tf.float32):
        stddev = tf.sqrt(2. / (shape[0] + shape[1]))
        return tf.random.normal(shape, stddev=stddev, dtype=dtype)

    def get_config(self):
        return {}

# Custom Activation Function


def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1, activation=my_softplus,
                          kernel_initializer=MyGlorotInitializer(),
                          kernel_regularizer=MyL1Regularizer(),
                          kernel_constraint=MyPositiveWeights())
])

model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

# Fit model
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

# Save model
model.save("saved_models/my_model_with_many_custom_parts.keras")

# Load model
model = tf.keras.models.load_model(
    "saved_models/my_model_with_many_custom_parts.keras",
    custom_objects={
        "MyL1Regularizer": MyL1Regularizer,
        "MyPositiveWeights": MyPositiveWeights,
        "MyGlorotInitializer": MyGlorotInitializer,
        "my_softplus": my_softplus,
    }
)

# Continue training
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.2008 - mae: 0.9706 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - loss: 1.1249 - mae: 0.6767 - val_loss: inf - val_mae: inf
Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step - loss: 0.8456 - mae: 0.5999 - val_loss: 2.8737 - val_mae: 0.5737
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - loss: 0.6909 - mae: 0.5547 - val_loss: 2.0950 - val_mae: 0.5363


### Custom Metrics


In [31]:
# extra code – once again, lets' create a basic Keras model
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 575us/step - huber_fn: 1.0788 - loss: 2.5942
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - huber_fn: 0.3403 - loss: 0.7644


#### Streaming metrics


In [32]:
precision = tf.keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [33]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [34]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [35]:
precision.variables

[<KerasVariable shape=(1,), dtype=float32, path=precision/true_positives>,
 <KerasVariable shape=(1,), dtype=float32, path=precision/false_positives>]

clear stored accumulated variables


In [36]:
precision.reset_state()
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

Creating a streaming metric:


In [92]:
tf.keras.backend.clear_session()

In [126]:

@keras.utils.register_keras_serializable()
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)  # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight(
            name='total', initializer='zeros', shape=(), dtype=tf.float32)
        self.count = self.add_weight(
            name='count', initializer='zeros', shape=(), dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [127]:
m = HuberMetric(2.)

# total = 2 * |10 - 2| - 2²/2 = 14
# count = 1
# result = 14 / 1 = 14
m(tf.constant([[2.]]), tf.constant([[10.]]))

<tf.Tensor: shape=(), dtype=float32, numpy=14.0>

In [128]:
# total = total + (|1 - 0|² / 2) + (2 * |9.25 - 5| - 2² / 2) = 14 + 7 = 21
# count = count + 2 = 3
# result = total / count = 21 / 3 = 7
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [129]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [130]:
print("Total:", m.total.numpy())  # This should print 21.0
print("Count:", m.count.numpy())  # This should print 3.0"

Total: 21.0
Count: 3.0


In [131]:
m.variables

[<KerasVariable shape=(), dtype=float32, path=huber_metric_4/total>,
 <KerasVariable shape=(), dtype=float32, path=huber_metric_4/count>]

In [132]:
m.reset_state()
m.variables

[<KerasVariable shape=(), dtype=float32, path=huber_metric_4/total>,
 <KerasVariable shape=(), dtype=float32, path=huber_metric_4/count>]

In [163]:
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

model.compile(loss=create_huber(2.0), optimizer="nadam",
              metrics=[HuberMetric(2.0)])

C:\Users\sayed\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [164]:
model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 568us/step - huber_metric_6: 1.0512 - loss: 1.0512
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - huber_metric_6: 0.3170 - loss: 0.3170


In [168]:
model.save("saved_models/my_model_with_a_custom_metric.keras")

In [169]:
model = tf.keras.models.load_model(
    "saved_models/my_model_with_a_custom_metric.keras",
    custom_objects={
        "huber_fn": create_huber(2.0),
        "HuberMetric": HuberMetric
    }
)

C:\Users\sayed\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 11 variables whereas the saved optimizer has 3 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [170]:
model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - huber_metric_6: 0.2588 - loss: 0.2588
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - huber_metric_6: 0.2270 - loss: 0.2270


In [190]:
try:
    model.metrics[-1].threshold
except Exception as ex:
    print(ex)

'CompileMetrics' object has no attribute 'threshold'


In [191]:
for metric in model.metrics:
    if isinstance(metric, HuberMetric):
        print(f"Huber Metric threshold: {metric.threshold}")

it seems like keras has updated from HOML tutorial, i will investigate where to find our threshold..


In [192]:
type(model.metrics[1])

keras.src.trainers.compile_utils.CompileMetrics

In [176]:
dir(model.metrics[-1])

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api_export_path',
 '_api_export_symbol_id',
 '_build_metrics_set',
 '_check_super_called',
 '_dtype',
 '_flat_metrics',
 '_flat_weighted_metrics',
 '_flatten_y',
 '_metrics',
 '_obj_type',
 '_tracker',
 '_unpickle_model',
 '_user_metrics',
 '_user_weighted_metrics',
 '_variables',
 'add_variable',
 'add_weight',
 'build',
 'built',
 'dtype',
 'from_config',
 'get_config',
 'metrics',
 'name',
 'output_names',
 'reset_state',
 'result',
 'stateless_reset_state',
 'stateless_result',
 'stateless_update_state',
 'update_state',
 'variables']

In [193]:
type(model.metrics[-1].metrics[0])

__main__.HuberMetric

In [194]:
model.metrics[-1].metrics[0].threshold

2.0

here it is :))))


Looks like it works fine! More simply, we could have created the class like this:


In [195]:
class HuberMetric(tf.keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [196]:
tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

model.compile(loss=tf.keras.losses.Huber(2.0), optimizer="nadam",
              weighted_metrics=[HuberMetric(2.0)])

np.random.seed(42)
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled, y_train, epochs=2,
                    sample_weight=sample_weight)

Epoch 1/2


C:\Users\sayed\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 593us/step - HuberMetric: 1.0599 - loss: 0.5274
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - HuberMetric: 0.3215 - loss: 0.1598


In [197]:
(history.history["loss"][0],
 history.history["HuberMetric"][0] * sample_weight.mean())

(0.3256884217262268, 0.32568849524955656)

In [198]:
model.save("saved_models/my_model_with_a_custom_metric_v2.keras")

In [199]:
model = tf.keras.models.load_model("saved_models/my_model_with_a_custom_metric_v2.keras",
                                   custom_objects={"HuberMetric": HuberMetric})

In [200]:
model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - HuberMetric: 0.2627 - loss: 0.2262
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - HuberMetric: 0.2298 - loss: 0.1999


In [204]:
model.metrics[-1].metrics[-1].threshold

2.0

### Custom Layers
